<a href="https://colab.research.google.com/github/KampHost/caldiss/blob/master/WhosFake_AKH.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#FakeTrump

Setup
1. Load libraries for preprocessing data
2. Load and preproces data

In [0]:
import pandas as pd
pd.set_option('display.max_colwidth', -1) #to see more text

import numpy as np
import spacy
nlp = spacy.load("en")

import itertools
from collections import Counter


In [0]:
FakeTrump = pd.read_csv('https://raw.githubusercontent.com/DeepLearnI/trump_tweet_classifier/master/code/tweet_labels.csv')

In [0]:
FakeTrump.info()

In [0]:
y = pd.DataFrame(pd.get_dummies(FakeTrump.labels)[0])

In [0]:
#Filter the corpus using tweet-preprocessor
! pip install tweet-preprocessor
import preprocessor as p
p.set_options(p.OPT.URL, p.OPT.EMOJI, p.OPT.HASHTAG)

In [0]:
#Clean up tweets
clean_tweet = FakeTrump['tweet'].map(p.clean)

In [0]:
clean_tweet=pd.DataFrame(clean_tweet)

In [0]:
# SPACY tokenizer
import spacy
nlp = spacy.load("en")

In [0]:
X = []
for tweet in nlp.pipe(clean_tweet['tweet']):
  FT_tok = [token.lemma_.lower() for token in tweet if token.pos_ in ['NOUN', 'PROPN', 'ADJ', 'ADV'] and not token.is_stop and not token.is_digit]
  X.append(FT_tok)

In [0]:
#Alternative Tokenizer 
# Tokenizing Tweets made easy!
#from nltk.tokenize import TweetTokenizer
#tknzr = TweetTokenizer()
#import nltk #this part is needed on colab.
#nltk.download('stopwords')
#from nltk.corpus import stopwords
#stop_words = set(stopwords.words('english'))

In [0]:
# Tokenize the tweets and remove stopwords
#train_tokens = train_df['review'].map(lambda t: [tok.lower() for tok in word_tokenize(t) if tok not in stop_words and tok.isalpha()])
#X = clean_tweet['tweet'].map(lambda textline: [tweet for tweet in tknzr.tokenize(textline) if tweet not in stop_words and tweet.isalpha()])

In [0]:
X

In [0]:
y['tokens'] = X

In [0]:
FakeTrump_new = y[y['tokens'].map(len) >0]

In [0]:
X=FakeTrump_new['tokens']

In [0]:
y=FakeTrump_new.iloc[:,0]

In [0]:
# Split the data using the train_test_split module. We keep 20% of the data for testing and use 80% to train the model
# Random state defined with an arbitrary number for reproducibility
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [0]:
X_train

In [0]:
y_train

In [0]:
X_test

In [0]:
y_test

In [0]:
!pip install -qq -U gensim
from gensim.corpora.dictionary import Dictionary

In [0]:
# Generate a dictionary
dictionary = Dictionary(X_train)

In [0]:
# construct corpus using this dictionary
train_corpus = [dictionary.doc2bow(doc) for doc in X_train]
test_corpus = [dictionary.doc2bow(doc) for doc in X_test]

In [0]:
# Tooling to map between corpus (gensim) and matrix - more general
from gensim.matutils import corpus2csc, corpus2dense

In [0]:
X_train = corpus2csc(train_corpus)
X_test = corpus2csc(test_corpus)

In [0]:
from sklearn.linear_model import LogisticRegression

In [0]:
model = LogisticRegression()
model.fit(X_train.T, y_train)

In [0]:
model.score(X_test.T, y_test)

0.7070234113712375